In [1]:
import os
import joblib
import pandas as pd
import numpy as np
import time
from scipy.sparse import load_npz
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier
# Modeller
from sklearn.feature_selection import SelectPercentile, f_classif 
from sklearn.svm import LinearSVC 

# --- KLASÖR AYARLARI ---
BASE_DIR = r"C:\Users\goktu\workspace\datacops"          # Okunacak yer (Ana klasör)
OUTPUT_DIR = os.path.join(BASE_DIR, "model_goktug")      # KAYDEDİLECEK YER (Senin klasörün)

# Klasörün var olduğundan emin olalım (Yoksa oluşturur)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Okuma Yeri: {BASE_DIR}")
print(f"Kaydetme Yeri: {OUTPUT_DIR}")
print("Ayarlar tamamlandı. ")

Okuma Yeri: C:\Users\goktu\workspace\datacops
Kaydetme Yeri: C:\Users\goktu\workspace\datacops\model_goktug
Ayarlar tamamlandı. 


In [2]:
print("Veriler yükleniyor...")
X_train_full = load_npz(os.path.join(BASE_DIR, "X_train_ready.npz"))
X_test_ready = load_npz(os.path.join(BASE_DIR, "X_test_ready.npz"))
feature_names = joblib.load(os.path.join(BASE_DIR, "feature_names.pkl"))

df_train = pd.read_csv(os.path.join(BASE_DIR, "train_scaled.csv"))
df_test = pd.read_csv(os.path.join(BASE_DIR, "test_scaled.csv"))
y_train_full = df_train["Crm Cd"].astype(str)
y_test = df_test["Crm Cd"].astype(str)

print(f"Veri Hazır. Boyut: {X_train_full.shape}")

Veriler yükleniyor...
Veri Hazır. Boyut: (848024, 1786)


In [3]:
SAMPLE_SIZE = 500000 

print(f"Hız için {SAMPLE_SIZE} satırlık örneklem alınıyor...")

try:
    X_train_bireysel, _, y_train_bireysel, _ = train_test_split(
        X_train_full, y_train_full, train_size=SAMPLE_SIZE, 
        stratify=y_train_full, random_state=42
    )
except ValueError:
    print("! Stratify olmadı, rastgele alınıyor...")
    X_train_bireysel, _, y_train_bireysel, _ = train_test_split(
        X_train_full, y_train_full, train_size=SAMPLE_SIZE, random_state=42
    )

print("Örneklem hazır.")

Hız için 500000 satırlık örneklem alınıyor...
Örneklem hazır.


In [4]:
print("1. Özellik Seçimi Yapılıyor (Tree-Based / Embedded Method)...")

# Seçim yapmak için basit bir Karar Ağacı kullanıyoruz
# Bu, modelin içine bakarak seçim yapan "Embedded" bir yöntemdir.
selection_model = DecisionTreeClassifier(max_depth=10, random_state=42)

# max_features=200: En önemli 200 özelliği seç diyoruz
selector_2 = SelectFromModel(selection_model, max_features=200, threshold=-np.inf)

# Seçimi küçük veri (sampling) üzerinde öğren
selector_2.fit(X_train_bireysel, y_train_bireysel)

# Verileri dönüştür (Sadece seçilenleri al)
X_train_sel_2 = selector_2.transform(X_train_bireysel)
X_test_sel_2 = selector_2.transform(X_test_ready)

print(f"Seçim Bitti ✅")
print(f"Seçilen Özellik Sayısı: {X_train_sel_2.shape[1]}")

1. Özellik Seçimi Yapılıyor (Tree-Based / Embedded Method)...
Seçim Bitti ✅
Seçilen Özellik Sayısı: 200


In [5]:
print("\n2. Model Eğitiliyor (Ridge Classifier)...")

# Ridge, L2 düzenlileştirme kullanan hızlı bir sınıflandırıcıdır.
clf_2 = RidgeClassifier(
    alpha=1.0, 
    random_state=42
)

start_time = time.time()
clf_2.fit(X_train_sel_2, y_train_bireysel)
end_time = time.time()

print(f"Eğitim Tamamlandı ({end_time - start_time:.2f} saniye) ✅")


2. Model Eğitiliyor (Ridge Classifier)...
Eğitim Tamamlandı (152.32 saniye) ✅


In [6]:
print("\n3. Tahmin Yapılıyor ve Dosya Oluşturuluyor...")

y_pred_2 = clf_2.predict(X_test_sel_2)

# Skorunu görelim (Model 1 ile karşılaştırmak için)
acc_2 = accuracy_score(y_test, y_pred_2)
print(f"\n 2. MODEL (RIDGE) SKORU: {acc_2 * 100:.2f}%")

# 2. Dosyayı Kaydet
submission_df_2 = pd.DataFrame({
    "DR_NO": df_test["DR_NO"],
    "Crm Cd": y_pred_2
})

filename_2 = "submission_goktug_model2.csv"
# Dosyayı senin klasörüne (OUTPUT_DIR) kaydediyoruz
save_path_2 = os.path.join(OUTPUT_DIR, filename_2)

submission_df_2.to_csv(save_path_2, index=False)

print(f"2. Dosya Başarıyla Kaydedildi: {filename_2}")
print(f"Konum: {save_path_2}")


3. Tahmin Yapılıyor ve Dosya Oluşturuluyor...

🔥 2. MODEL (RIDGE) SKORU: 33.74%
✅ 2. Dosya Başarıyla Kaydedildi: submission_goktug_model2.csv
📂 Konum: C:\Users\goktu\workspace\datacops\model_goktug\submission_goktug_model2.csv
